In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data=pd.read_csv('data_clean.csv',engine='python')
y=pd.read_csv('data.csv',engine='python')['status']

In [3]:
data.head()

,锘�,custid,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,...,3_y,4_y,5_y,6_y,7_y,8_y,9_y,10_y,11_y,12_y
0,0,0.697138,0.01,0.99,0.000000,0.000019,0.488636,0.308370,0.375,0.094737,...,0,1,0,0,0,0,0,0,0,0
1,1,0.133331,0.02,0.94,0.029412,0.000027,1.000000,0.468062,0.425,0.105263,...,0,0,1,0,0,0,0,0,0,0
2,2,0.711603,0.04,0.96,0.000000,0.000021,1.000000,0.089207,0.275,0.238596,...,0,0,1,0,0,0,0,0,0,0
3,3,0.451881,0.00,0.96,0.029412,0.000003,0.511364,0.819383,0.500,0.049123,...,0,0,1,0,0,0,0,0,0,0
4,4,0.624214,0.01,0.99,0.000000,0.000010,1.000000,0.156388,0.275,0.231579,...,0,0,0,0,0,0,0,0,0,0


In [4]:
for i in data.columns:
     if data[i].dtypes==object:
            print(i)

latest_query_time
loans_latest_time


In [5]:
data.drop(['latest_query_time','loans_latest_time'],axis=1,inplace=True)

In [6]:
from sklearn.model_selection import  train_test_split
x_train, x_test, y_train, y_test = train_test_split(data,y,train_size=0.7, random_state=2018)

H:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


###  随机森林挑选特征

In [7]:
from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier(n_estimators=20,random_state=2019,n_jobs=1)
forest.fit(x_train,y_train)
forest.score(x_test,y_test)

H:\Anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


0.7708479327259986

In [8]:
importance = forest.feature_importances_#重要特征是指与因变量密切相关的特征，并且对因变量的变化影响较大
imp_result = np.argsort(importance)[::-1]#排序

for i in range(len(x_train)-1):
    print("%2d. %-*s %f"%(i+1, 30, x_train.columns[i], importance[imp_result[i]]))

 1. 锘�                             0.047994
 2. custid                         0.041648
 3. low_volume_percent             0.040752
 4. middle_volume_percent          0.028679
 5. take_amount_in_later_12_month_highest 0.023547
 6. trans_amount_increase_rate_lately 0.020278
 7. trans_activity_month           0.018777
 8. trans_activity_day             0.017722
 9. transd_mcc                     0.015768
10. trans_days_interval_filter     0.015362
11. trans_days_interval            0.015343
12. regional_mobility              0.014567
13. repayment_capability           0.014521
14. is_high_user                   0.014196
15. number_of_trans_from_2011      0.014074
16. first_transaction_time         0.013983
17. historical_trans_amount        0.013756
18. historical_trans_day           0.013596
19. rank_trad_1_month              0.013580
20. trans_amount_3_month           0.013518
21. avg_consume_less_12_valid_month 0.013431
22. abs                            0.013412
23. top_trans_count_l

IndexError: index 116 is out of bounds for axis 0 with size 116

In [9]:
threshold = 0.01
data_index = list(x_train.columns[ importance < threshold])
x_train.drop(data_index,axis=1,inplace=True)
x_test.drop(data_index,axis = 1,inplace = True)

H:\Anaconda\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [10]:
from sklearn import ensemble
model_random_forest_Classifier = ensemble.RandomForestClassifier(n_estimators=20,random_state = 2019)
model_random_forest_Classifier.fit(x_train,y_train)
model_random_forest_Classifier.score(x_test,y_test)

0.7687456201822004

### IV值（借鉴）

In [ ]:
def CalcIV(Xvar,Yvar):
    N_0=np.sum(Yvar==0)
    N_1=np.sum(Yvar==1)
    N_0_group=np.zeros(np.unique(Xvar).shape)
    
    N_1_group=np.zeros(np.unique(Xvar).shape)
    for i in range(len(np.unique(Xvar))):
        N_0_group[i] = Yvar[(Xvar==np.unique(Xvar)[i])&(Yvar==0)].count()
        N_1_group[i] = Yvar[(Xvar==np.unique(Xvar)[i])&(Yvar==1)].count()
    iv = np.sum((N_0_group/N_0-N_1_group/N_1)*np.log((N_0_group/N_0)/(N_1_group/N_1)))
    if iv>=1.0:## 处理极端值
        iv=1
    return iv

def caliv_batch(df,Yvar):
    ivlist=[]
    for col in df.columns:
        iv=CalcIV(df[col],Yvar)
        ivlist.append(iv)
    names=list(df.columns)
    iv_df=pd.DataFrame({'Var':names,'Iv':ivlist},columns=['Var','Iv'])

    return iv_df,ivlist
im_iv, ivl = caliv_batch(data,y)


H:\Anaconda\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
H:\Anaconda\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.
